In [ ]:
# default_exp problem_types.seq_tag
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
# test setup
import tensorflow as tf
import numpy as np
from m3tl.test_base import TestBase
from m3tl.input_fn import train_eval_input_fn
from m3tl.test_base import test_top_layer
test_base = TestBase()
params = test_base.params

hidden_dim = params.bert_config.hidden_size

train_dataset = train_eval_input_fn(params=params)
one_batch = next(train_dataset.as_numpy_iterator())


# Sequence Labeling(seq_tag)

This module includes neccessary part to register sequence labeling problem type.

## Imports and utils


In [ ]:
# export
from typing import List

import numpy as np
import tensorflow as tf
from m3tl.base_params import BaseParams
from m3tl.problem_types.utils import (create_dummy_if_empty,
                                      empty_tensor_handling_loss,
                                      nan_loss_handling)
from m3tl.special_tokens import PREDICT, TRAIN
from m3tl.utils import (LabelEncoder, get_label_encoder_save_path, get_phase,
                        need_make_label_encoder, variable_summaries)
from tensorflow_addons.layers.crf import CRF
from tensorflow_addons.text.crf import crf_log_likelihood


## Top Layer

In [ ]:
# export

class SequenceLabel(tf.keras.Model):
    def __init__(self, params: BaseParams, problem_name: str):
        super(SequenceLabel, self).__init__(name=problem_name)
        self.params = params
        self.problem_name = problem_name
        num_classes = self.params.get_problem_info(problem=problem_name, info_name='num_classes')
        self.dense = tf.keras.layers.Dense(num_classes, activation=None)

        self.dropout = tf.keras.layers.Dropout(1-params.dropout_keep_prob)

        if self.params.crf:
            self.crf = CRF(num_classes)
            self.metric_fn = tf.keras.metrics.Accuracy(
                name='{}_acc'.format(self.problem_name)
            )
        else:
            self.metric_fn = tf.keras.metrics.SparseCategoricalAccuracy(
                name='{}_acc'.format(self.problem_name))

    def return_crf_result(self, labels: tf.Tensor, logits: tf.Tensor, mode: str, input_mask: tf.Tensor):
        input_mask.set_shape([None, None])
        logits = create_dummy_if_empty(logits)
        input_mask = create_dummy_if_empty(input_mask)
        viterbi_decoded, potentials, sequence_length, chain_kernel = self.crf(
            logits, input_mask)
        if mode != PREDICT:
            loss = -crf_log_likelihood(potentials,
                                       labels, sequence_length, chain_kernel)[0]
            loss = tf.reduce_mean(loss)
            loss = nan_loss_handling(loss)
            self.add_loss(loss)
            acc = self.metric_fn(
                labels, viterbi_decoded, sample_weight=input_mask)
            self.add_metric(acc)

        # make the crf prediction has the same shape as non-crf prediction
        return tf.one_hot(viterbi_decoded, name='%s_predict' % self.problem_name, depth=self.params.num_classes[self.problem_name])

    def call(self, inputs):
        mode = get_phase()
        training = (mode == TRAIN)
        feature, hidden_feature = inputs
        hidden_feature = hidden_feature['seq']
        if mode != PREDICT:
            labels = feature['{}_label_ids'.format(self.problem_name)]
            # sometimes the length of labels dose not equal to length of inputs
            # that's caused by tf.data.experimental.bucket_by_sequence_length in multi problem scenario
            pad_len = tf.shape(input=hidden_feature)[
                1] - tf.shape(input=labels)[1]

            # top, bottom, left, right
            pad_tensor = [[0, 0], [0, pad_len]]
            labels = tf.pad(tensor=labels, paddings=pad_tensor)

        else:
            labels = None
        hidden_feature = self.dropout(hidden_feature, training)

        if self.params.crf:
            return self.return_crf_result(labels, hidden_feature, mode, feature['model_input_mask'])

        logits = self.dense(hidden_feature)

        if self.params.detail_log:
            for weight_variable in self.weights:
                variable_summaries(weight_variable, self.problem_name)

        if mode != PREDICT:
            loss = empty_tensor_handling_loss(
                labels, logits,
                tf.keras.losses.sparse_categorical_crossentropy)
            self.add_loss(loss)
            acc = self.metric_fn(
                labels, logits, sample_weight=feature['model_input_mask'])
            self.add_metric(acc)
        return tf.nn.softmax(
            logits, name='%s_predict' % self.problem_name)

In [ ]:
test_top_layer(SequenceLabel, problem='weibo_fake_ner', params=params, sample_features=one_batch, hidden_dim=hidden_dim)
params.crf = False
test_top_layer(SequenceLabel, problem='weibo_fake_ner', params=params, sample_features=one_batch, hidden_dim=hidden_dim)

## Get or make label encoder function


In [ ]:
# export


def seq_tag_get_or_make_label_encoder_fn(params: BaseParams, problem: str, mode: str, label_list: List[str], *args, **kwargs) -> LabelEncoder:

    le_path = get_label_encoder_save_path(params=params, problem=problem)
    label_encoder = LabelEncoder()

    if need_make_label_encoder(mode=mode, le_path=le_path, overwrite=kwargs['overwrite']):
        # fit and save label encoder
        label_list = [item for sublist in label_list for item in sublist] + ['[PAD]']
        label_encoder.fit(label_list)
        label_encoder.dump(le_path)
        params.set_problem_info(problem=problem, info_name='num_classes', info=len(label_encoder.encode_dict))
    else:
        label_encoder.load(le_path)

    return label_encoder

## Label handing function

In [ ]:
# export
def seq_tag_label_handling(tokenized_dict: dict, target: list, pad_token: str) -> tuple:
    special_token_mask = tokenized_dict['special_tokens_mask']
    del tokenized_dict['special_tokens_mask']

    # handle truncation
    if tokenized_dict.get('num_truncated_tokens', 0) > 0:
        target = target[:len(target) - tokenized_dict['num_truncated_tokens']]

    processed_target = []
    for m in special_token_mask:
        # 0 is special tokens, 1 is tokens
        if m == 1:
            # add pad
            processed_target.append(pad_token)
        else:
            processed_target.append(target.pop(0))
    return processed_target, tokenized_dict

def seq_tag_label_handling_fn(target, label_encoder=None, tokenizer=None, decoding_length=None, *args, **kwargs):

    if kwargs is None:
        return {}

    if 'tokenized_inputs' not in kwargs:
        return {}

    if kwargs['tokenized_inputs'] is None:
        return {}

    tokenized_inputs = kwargs['tokenized_inputs']
    # target should align with input_ids
    target, tokenized_inputs = seq_tag_label_handling(
        tokenized_inputs, target, '[PAD]')

    if len(target) != len(tokenized_inputs['input_ids']):
        raise ValueError(
            'Length is different for seq tag problem, inputs: {}'.format(tokenizer.decode(tokenized_inputs['input_ids'])))

    label_id = label_encoder.transform(target).tolist()
    label_id = [np.int32(i) for i in label_id]
    return label_id, None

